<a href="https://colab.research.google.com/github/barcellfe/datascience_study/blob/master/Aula_04_11_2020_Pandas_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Retomando: 

Aprendemos na aula anterior a manipular de forma mais precisa alguns elementos dos índices, fazer reíndice, manipular linhas e colunas usando loc e iloc.

Lembrando que loc serve para encontrar índices com texto
E que iloc serve para encontrar índices numéricos

Vimos também que a função apply e applymap são úteis para se fazer rotinas que varrem um dataframe inteiro.

A aula de hoje contempla os capítulos 7 e 8 do livro Python para análise de dados. Onde iniciaremos as etapas de limpeza de dados e manipulação básica de tabelas e joins/merges.

#1. Lidando com dados vazios:

Vimos anteriormente no numpy a função np.isnan. No Pandas, utilizaremos a função isnull()

In [2]:
import pandas as pd
import numpy as np

In [3]:
string_data = pd.Series(['aadvark', 'artichoke', np.nan, 'avocado'])
string_data

0      aadvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [5]:
##Palavra reservada nova, None
string_data[0] = None
string_data

0         None
1    artichoke
2          NaN
3      avocado
dtype: object

In [6]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

Na tabela 7.1 do livro, temos os métodos de remoção de NA:

1. dropna
2. fillna
3. isnull
4. notnul (equivalente a um ~isnull)

In [7]:
string_data.dropna() ##observem a facilidade

1    artichoke
3      avocado
dtype: object

In [8]:
string_data = string_data.fillna('No Data')
string_data

0      No Data
1    artichoke
2      No Data
3      avocado
dtype: object

In [9]:
from numpy import nan as NA

data = pd.Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [ ]:
##removendo os valores

data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [ ]:
##Esse comando é equivalente ao anterior
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [10]:
##Agora vamos trabalhar com a estrutura de nulos considerando dataframes e não séries
##detalhe na forma de declaração do dataframe
data = pd.DataFrame([[1., 6.5, 3], [1., NA, NA], [NA, NA, NA],[NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [11]:
##vamos ver a fundo cada método de remoção de NA's

cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


In [12]:
##usando o argumento how = 'all', apenas linhas completas são removidas

data.dropna(how = 'all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [13]:
##Para atingir as colunas, basta usar axis = 1

data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [14]:
data.dropna(how = 'all', axis = 1)

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [ ]:
##o argumento thresh limita o número de Na's que podem ser removidos de uma só vez

##em casos de séries de tempo, pode prejudicar a estrutura de dados

In [15]:
data.dropna(thresh=1)

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [16]:
# Testando a qtd a ser removida
data.dropna(thresh=2)

,0,1,2,4
0,1.0,6.5,3.0,NaN
3,NaN,6.5,3.0,NaN


#2. Preenchendo Dados Vazios

Nem só de remoção vive o cientista de dados. Muitas vezes, precisamos preencher os NAs com alguns valores.

In [17]:
# gerando dados com NaN
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,1.160187,NaN,NaN
1,1.443159,NaN,NaN
2,0.591961,NaN,2.094239
3,0.644615,NaN,1.218934
4,-0.521813,0.238089,-1.124089
5,0.491251,-0.921325,2.095000
6,0.987804,0.684464,-0.648235


In [18]:
##preenchendo os NaN com 0

df.fillna(0)

,0,1,2
0,1.160187,0.000000,0.000000
1,1.443159,0.000000,0.000000
2,0.591961,0.000000,2.094239
3,0.644615,0.000000,1.218934
4,-0.521813,0.238089,-1.124089
5,0.491251,-0.921325,2.095000
6,0.987804,0.684464,-0.648235


In [20]:
##cada coluna pode ser preenchida com um valor diferente

df.fillna({1: 0.5, 2:0})

,0,1,2
0,1.160187,0.500000,0.000000
1,1.443159,0.500000,0.000000
2,0.591961,0.500000,2.094239
3,0.644615,0.500000,1.218934
4,-0.521813,0.238089,-1.124089
5,0.491251,-0.921325,2.095000
6,0.987804,0.684464,-0.648235


In [21]:
##para a regra do fill na, também podemos declarar com a regra do inplace

df.fillna(0, inplace = True)
df

,0,1,2
0,1.160187,0.000000,0.000000
1,1.443159,0.000000,0.000000
2,0.591961,0.000000,2.094239
3,0.644615,0.000000,1.218934
4,-0.521813,0.238089,-1.124089
5,0.491251,-0.921325,2.095000
6,0.987804,0.684464,-0.648235


In [22]:
##fillna também usa os métodos de ffill e bfill:

df = pd.DataFrame(np.random.randn(6,3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,-1.315898,-0.580217,-1.309756
1,-1.128046,-0.882092,-0.647461
2,1.718062,NaN,0.777871
3,0.248026,NaN,0.090282
4,-0.440341,NaN,NaN
5,0.727543,NaN,NaN


In [23]:
df.fillna(method = 'ffill')

,0,1,2
0,-1.315898,-0.580217,-1.309756
1,-1.128046,-0.882092,-0.647461
2,1.718062,-0.882092,0.777871
3,0.248026,-0.882092,0.090282
4,-0.440341,-0.882092,0.090282
5,0.727543,-0.882092,0.090282


In [24]:
# Voltando os NaN
df.fillna(method = 'bfill') ##só funciona quando os valores são anteriores ao NA

,0,1,2
0,-1.315898,-0.580217,-1.309756
1,-1.128046,-0.882092,-0.647461
2,1.718062,NaN,0.777871
3,0.248026,NaN,0.090282
4,-0.440341,NaN,NaN
5,0.727543,NaN,NaN


In [25]:
df.fillna(method = 'ffill', limit = 2) ##limitado a dois preenchimentos

,0,1,2
0,-1.315898,-0.580217,-1.309756
1,-1.128046,-0.882092,-0.647461
2,1.718062,-0.882092,0.777871
3,0.248026,-0.882092,0.090282
4,-0.440341,NaN,0.090282
5,0.727543,NaN,0.090282


In [26]:
df.fillna(df.mean()) ##usa a média para preencher buracos

,0,1,2
0,-1.315898,-0.580217,-1.309756
1,-1.128046,-0.882092,-0.647461
2,1.718062,-0.731154,0.777871
3,0.248026,-0.731154,0.090282
4,-0.440341,-0.731154,-0.272266
5,0.727543,-0.731154,-0.272266


In [ ]:
##Tabela 7.2 Métodos específicos para fillna

#3. Transformação de Dados

In [27]:
##Removendo duplicatas

data = pd.DataFrame({'k1': ['one', 'two'] *3 +['two'], ##replica one two 3x e o ultimo é um two
                     'k2': [1,1,2,3,3,4,4]})

data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [28]:
data.duplicated() ##check de duplicatas

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [29]:
data.drop_duplicates() ##remoção de duplicatas

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [30]:
data['v1'] = range(7) ##criamos um vetor dummy
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [31]:
data.drop_duplicates(['k1']) ##remove na coluna especificada

,k1,k2,v1
0,one,1,0
1,two,1,1


In [32]:
data.drop_duplicates(['k1', 'k2'], keep = 'last') ##em caso de empate, escolhe quem fica

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


In [ ]:
##Manipulando dados usando funções ou applymap

In [33]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [34]:
##criando um mapa

meat_to_animal = {'bacon': 'pig',
                  'pulled pork': 'pig',
                  'pastrami': 'cow',
                  'corned beef': 'cow',
                  'honey ham': 'pig',
                  'nova lox': 'salmon'}

meat_to_animal

{'bacon': 'pig',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon',
 'pastrami': 'cow',
 'pulled pork': 'pig'}

In [35]:
##métodos str

lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [36]:
data['animal'] = lowercased.map(meat_to_animal) ##percebam a interação da função map nessa linha de código
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [ ]:
##Modificando valores com o comando replace

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000. ,3.])

data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [ ]:
data.replace(-999., NA)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [ ]:
##podemos retirar multiplas colunas usando uma lista

data.replace([-999., -1000.], NA)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [ ]:
##podemos substituir multiplos valores

data.replace([-999., -1000.], [NA, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [ ]:
##podemos substituir como um dict

data.replace({-999. : NA, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [37]:
##Modificando os índices dos eixos

data = pd.DataFrame(np.arange(12).reshape(3,4),
                    index = ['Ohio', 'Colorado', 'New York'],
                    columns = ['one', 'two', 'three', 'four'])

data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [38]:
transform = lambda x: x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [39]:
data.index = data.index.map(transform)

data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [ ]:
##Também podemos usar a função rename

data.rename(index = str.title, columns = str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [ ]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [ ]:
data.rename(index = {'OHIO': 'INDIANA'},
            columns = {'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [ ]:
##Também podemos aplicar a função inplace

data.rename(index = {'OHIO': 'INDIANA'}, inplace = True)

# 4. Manipulação de Strings

In [ ]:
val = 'a, b, guido'
val

'a, b, guido'

In [ ]:
##separando a string em objetos de uma lista

val.split(',')

['a', ' b', ' guido']

In [ ]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [ ]:
first, second, third = pieces
pieces

['a', 'b', 'guido']

In [ ]:
first + '::' + second + '::' + third ##parece muito com a função concat do excel

'a::b::guido'

In [ ]:
##podemos fazer a mesma coisa usando o comando join

'::'.join(pieces)

'a::b::guido'

In [ ]:
'guido' in val

True

In [ ]:
val.index(',')

1

In [ ]:
val.find(':')

-1

In [ ]:
val.index(':')

ValueError: ignored

In [ ]:
val.count(',')

2

In [ ]:
val.replace(',','::')

'a:: b:: guido'

In [ ]:
val.replace(',', '')

'a b guido'

In [ ]:
##Para a próxima semana: resumo das funções da tabela 7.3 para manipulação de strings

In [ ]:
##Tópico Importante: Regex

In [ ]:
import re
text = 'foo        bar\t   baz     \tqux'
text

'foo        bar\t   baz     \tqux'

In [ ]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [ ]:
regex = re.compile('\s+')
regex

re.compile(r'\s+', re.UNICODE)

In [ ]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [ ]:
regex.findall(text)

['        ', '\t   ', '     \t']

In [40]:
##exemplos

text = """ Dave dave@google.com
           Steve steve@gmail.com
           Rob rob@gmail.com
           Ryan ryan@yahoo.com
      """

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}' ##padrao para pegar um email

In [41]:
regex = re.compile(pattern, flags = re.IGNORECASE)
regex.findall(text)

NameError: ignored

In [ ]:
m = regex.search(text)
m

<_sre.SRE_Match object; span=(6, 21), match='dave@google.com'>

In [ ]:
text[m.start():m.end()]

'dave@google.com'

In [ ]:
print(regex.match(text)) ##só retorna se for no começo do texto

None


In [ ]:
##alterando valores

print(regex.sub('CENSURADO', text)) ##substitui onde dá match

 Dave CENSURADO
           Steve CENSURADO
           Rob CENSURADO
           Ryan CENSURADO
      


In [ ]:
##dividindo o email

pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9._]+)\.([A-Z]{2,4})'

regex = re.compile(pattern, flags = re.IGNORECASE)

m = regex.match('wes@bright.net')
m

<_sre.SRE_Match object; span=(0, 14), match='wes@bright.net'>

In [ ]:
m.groups()

('wes', 'bright', 'net')

In [ ]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [ ]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

 Dave Username: dave, Domain: google, Suffix: com
           Steve Username: steve, Domain: gmail, Suffix: com
           Rob Username: rob, Domain: gmail, Suffix: com
           Ryan Username: ryan, Domain: yahoo, Suffix: com
      


In [ ]:
##Ver tabela 7.4 para métodos de regex